In [104]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [105]:
# read csv files
hawaii_measurements = 'clean_hawaii_measurements.csv'
hawaii_stations = 'clean_hawaii_stations.csv'

measurements_data = pd.read_csv(hawaii_measurements)
stations_data = pd.read_csv(hawaii_stations)

In [106]:
# view measurements_data
measurements_data.head()

,Unnamed: 0,station,date,prcp,tobs
0,4,USC00519397,2010-01-06,0.0,73
1,26,USC00519397,2010-01-30,0.0,70
2,29,USC00519397,2010-02-03,0.0,67
3,43,USC00519397,2010-02-19,0.0,63
4,61,USC00519397,2010-03-11,0.0,73


In [107]:
# view stations_data
stations_data.head()

,Unnamed: 0,station,name,latitude,longitude,elevation
0,0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


## Database Creation

In [108]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [109]:
# Create measurements and stations classes
class Measurements(Base):
    __tablename__ = "measurements"
    id = Column(Integer, primary_key = True)
    station = Column(String(255))
    date = Column(Integer)
    prcp = Column(Integer)
    tobs = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}" 
    
# stations 
class Stations(Base):
    __tablename__ = "stations"
    id = Column(Integer, primary_key = True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Integer)
    longitude = Column(Integer)
    elevation = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [110]:
# Create Engine and Pass in MySQL Connection
engine = create_engine("sqlite:///hawaii.sqlite")
conn = engine.connect()

# Use `create_all` to create the demographics table in the database
Base.metadata.create_all(engine)

In [111]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
measurements_db = measurements_data.to_dict(orient="records")
stations_db = stations_data.to_dict(orient="records")

In [112]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

# Save the reference to the `demographics` table as a variable called `table`
table_measurements = sqlalchemy.Table('Measurements', metadata, autoload=True)
table_stations = sqlalchemy.Table('Stations', metadata, autoload=True)

In [113]:
# Use `table.insert()` to insert the data into the table
conn.execute(table_measurements.insert(), measurements_db)
conn.execute(table_stations.insert(), stations_db)

In [114]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from Measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-06', 0, 73),
 (2, 'USC00519397', '2010-01-30', 0, 70),
 (3, 'USC00519397', '2010-02-03', 0, 67),
 (4, 'USC00519397', '2010-02-19', 0, 63),
 (5, 'USC00519397', '2010-03-11', 0, 73)]

In [115]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from Stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]